
In this course, you will learn about:

sqlite - interacting with SQLite databases;
xml - creating and processing XML files;
csv - CSV file reading and writing;
logging - basics logging facility for Python;
configparser - configuration file parser.





# SQLITE

- connect - method
- cursor - method of connect obj

When connecting to the database using the connect method, a Connection object is created. It has a very useful method called cursor. The method creates a Cursor object that allows any SQL statements to be executed in the database. What does it look like in practice? Let's look at the code responsible for creating the table called tasks:


- execute - Calling the execute method executes the CREATE TABLE statement in our database. The execute method takes any single SQL statement and optional parameters necessary to execute the query. The variant with optional parameters will be presented when we discuss inserting data in the database.


NOTE: Running the above program twice will throw an exception with the following message: sqlite3.OperationError: table tasks already exists. This is because the statement is trying to re-create a table with the same name. The solution to this problem is to modify the query as follows:


types :
INTEGER, TEXT, NULL, REAL, BLOB

INSERT INTO

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS  tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')


commit()
close()

NOTE: Restarting the program will create another task with the same name and priority, but with a different value that is auto-incremental.



executemany() - The executemany method allows you to insert multiple records at once. As an argument, it accepts an SQL statement and an array containing any number of tuples.

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')
tasks = [
    ('My first task', 1),
    ('My second task', 5),
    ('My third task', 10),
]
c.executemany('INSERT INTO tasks (name, priority) VALUES (?,?)', tasks)
conn.commit()
conn.close()


__Refatcoring__ - Refactoring is a very important process during software development. The main purpose of refactoring is to improve the quality of the code. Every programmer in their career will have to refactor either their own or someone else’s code sooner or later.

A very common mistake made by young adepts of the art of programming is to repeat the same pieces of code in different places in the application. In this case, refactoring consists of creating a function containing repetitive fragments. As a result, the code’s volume is reduced, and it becomes more readable.

You've probably noticed that adding new functionalities to our TODO application would be very difficult. This is a sign that our application requires refactoring. Below are suggestions for changes we can make:

1. creating a class called TODO that will connect to the database in the constructor. If you want, you can implement a separate method called connect for this purpose;
2. moving the code responsible for creating the taskstable to the method named create_tasks_table;
3. creating a method called add_task that will get the task name and priority from the user instead of using hardcoded values.

Will we be able to easily implement, for example, the data display functionality after these changes? Find out about this later in the course.



In [ ]:
import sqlite3

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def add_task(self):
        name = input('Enter task name: ')
        priority = int(input('Enter priority: '))
        
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
        self.conn.commit()

app = Todo()
app.add_task()

__SELECT__

The SELECT statement allows you to read data from one or more tables. Its syntax looks like this:

SELECT column FROM table_name;

or

SELECT column1, column2, column3, …, columnN FROM table_name;

or

SELECT * FROM table_name;


In the first variant, we decide to read the values saved in only one column. If we'd like to read only the names of the tasks saved in the tasks table, we could use the following query:

SELECT name FROM tasks;

The second variant allows you to read values from more columns. If we'd like to read the task names and their priorities, we could use the following query:

SELECT name, priority FROM tasks;

If we don’t have any specific requirements, we can read the values from all columns:

SELECT * FROM tasks;

The last variant will display the values saved in the id, name and priority columns.



In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
for row in c.execute('SELECT * FROM tasks'):
    print(row)
conn.close()


- It probably won’t surprise you to learn that reading data saved in the database is done with the well-known Cursor object. After calling the execute method with the appropriate SELECT statement, the Cursor object is treated as an iterator. Look at the code in the editor.


!!! The variable row in each iteration takes a row in the form of a tuple. Access to individual columns is done using an index, e.g., print (row [0]) will display the values saved in the id column.

__fetchall__ The fetchall method fetches all records (those not yet fetched from the query result). Look at the code in the editor.



!!!! The fetchall method is less efficient than the iterator, because it reads all records into the memory and then returns a list of tuples. For small amounts of data, it doesn't matter, but if your table contains a huge number of records, this can cause memory issues.

NOTE: The fetchall method returns an empty list when no rows are available.

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('SELECT * FROM tasks')
rows = c.fetchall()
for row in rows:
    print(row)
conn.close()


__fetchone__ 

the Cursor object provides a very useful method called fetchone to retrieve the next available record. Look at the code in the editor.

Result:

(1, 'My first task', 1)
(2, 'My second task', 5)

NOTE: The fetchone method returns None if there is no data to read.

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('SELECT * FROM tasks')
row = c.fetchone()
print(row)
row = c.fetchone()
print(row)
conn.close()


LABO : 1.2.1.1

__UNIQUE__ 

If you never want to have duplicates, you should declare this as a table constraint:

CREATE TABLE bookmarks(
    users_id INTEGER,
    lessoninfo_id INTEGER,
    UNIQUE(users_id, lessoninfo_id)

INSERT OR IGNORE INTO bookmarks(users_id, lessoninfo_id) VALUES(123, 456)

__UPDATE__
- Each of the tasks created has its own priority, but what if we decide that one of them should be done earlier than the others. How can we increase its priority? We have to use the SQL statement called UPDATE.


As before, we execute all SQL statements using the execute method. Look at the code in the editor.

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('UPDATE tasks SET priority = ? WHERE id = ?', (20, 1))
c.commit()
c.close()


If we'd like to set the priority to 20 for a task with idequal to 1, we can use the following query:

UPDATE tasks SET priority = 20 WHERE id = 1;

!!! NOTE: If you forget about the WHERE clause, all data in the table will be updated.

__DELETE__ 

schemat = DELETE FROM table_name WHERE condition;

In [ ]:
import sqlite3

conn = sqlite3.connect('todo.db')
c = conn.cursor()
c.execute('DELETE FROM tasks WHERE id = ?', (1,))
c.commit()
c.close()


LAB 1.4.1.1

In [ ]:
import sqlite3

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def add_task(self):
        name = input('Enter task name: ')
        priority = int(input('Enter priority: '))
        
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
        self.conn.commit()

    def find_task(self,name_of_task = None):

        name_of_task = input('Enter task name:')
        id = int(input('Enter priority:'))


        for row in c.execute('SELECT * FROM tasks'):
            if row[0] == id and row[1] == name_of_task:
                print(row)


app = Todo()
app.add_task()










